from PIL import Image
np.array(Image.open(VALIDATION_DIR+'0001eeaf4aed83f9.jpg')).shape

In [1]:
from cfg import *
from utils.generator import *
from model.rpn import *
# from model.head import *

from keras import layers, models
from keras import initializers
from keras import regularizers
import keras.backend as K

import tensorflow as tf

gen = OIDGen('split_1_annotation.json', batch_size=1)
img_size = gen[0][0][0].shape.as_list()[1:3]

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [2]:
from keras.applications import VGG16        
vgg16 = VGG16(include_top=False, )
backbone = models.Model(vgg16.input, vgg16.get_layer('block5_conv3').output)

x = backbone(gen[0][0])
# initial rpn model
rpn = RPN(img_size)(x)

proposals, scores = rpn(x)

In [11]:
class RoiPooling(layers.Layer):
    def __init__(self, pool_width=7, pool_height=7, ):
        super(RoiPooling, self).__init__()
        self.pool_width = pool_width
        self.pool_height = pool_height

    def call(self, inp):
        # x (1, 16, 16, 512)
        # rois (N, 4)
        x, rois = inp

        x_shape = x.shape.as_list()[1:3][::-1]
        scale = tf.Variable(x_shape+x_shape, dtype=tf.float32)        
        rois /= scale
        rois_pool = tf.image.crop_and_resize(x,rois, tf.zeros([tf.shape(rois)[0]], dtype=tf.int32), [self.pool_height, self.pool_width])        
#         rois_pool = tf.map_fn(self.helper, rois)

        return rois_pool     
    
    def compute_output_shape(self, inp):
        x, rois = inp
        return (1,None)+ x[1:]
    
    def helper(self, inp):
        x_shape = x.shape.as_list()[1:3][::-1]
        scale = tf.Variable(x_shape+x_shape, dtype=tf.float32)
        roi = tf.cast(inp[0] / scale, tf.int32)
        roi_pool = tf.image.resize_images(x[0,roi[0]:roi[2], roi[1]:roi[3],:], [self.pool_height,self.pool_width],)
        return roi_pool        

In [68]:
class RoiAlian(layers.Layer):
    def __init__(self, pool_height=7, pool_width=7):
        super(RoiAlian, self).__init__()
        self.pool_height = pool_height
        self.pool_width = pool_width

    def call(self, inp,):
        x, rois = inp
        x = tf.pad(x, [[0,0],[1,1],[1,1],[0,0]], mode='SYMMETRIC')
        rois += 1

        x0,y0,x1,y1 = tf.split(rois, 4, 1)
        w = x1 - x0
        h = y1 - y0

        xx = (x0 + w/2/self.pool_width - .5) / tf.to_float(tf.shape(x)[2])
        yy = (y0 + h/2/self.pool_height - .5) / tf.to_float(tf.shape(x)[1])

        w /= tf.to_float(tf.shape(x)[2])
        h /= tf.to_float(tf.shape(x)[1])
        rois =  tf.concat([xx, yy, xx+w, yy+h], axis=1)
        roi_pools = tf.image.crop_and_resize(x, rois, tf.zeros([tf.shape(rois)[0]], dtype=tf.int32), 
                                [self.pool_width, self.pool_height])
        roi_pools = layers.AveragePooling2D(strides=2 , padding="same")(roi_pools)
        return roi_pools

    def compute_output_shape(self, inp):
        x, rois = inp
        return (None,) + x[1:]

In [20]:
x, proposals, scores

(<tf.Tensor 'model_1/block5_conv3/Relu:0' shape=(1, 64, 30, 512) dtype=float32>,
 <tf.Tensor 'model_2/proposal_layer_1/GatherV2_3:0' shape=(?, 4) dtype=float32>,
 <tf.Tensor 'model_2/proposal_layer_1/GatherV2_4:0' shape=(?,) dtype=float32>)

In [69]:
roi = RoiAlian()([x,proposals])

In [70]:
roi

<tf.Tensor 'roi_alian_9/average_pooling2d_3/AvgPool:0' shape=(?, 4, 4, 512) dtype=float32>

In [44]:
layers.Flatten()(roi)

<tf.Tensor 'flatten_1/Reshape:0' shape=(?, ?) dtype=float32>

In [60]:
x_, x_1, x_2

(<tf.Tensor 'model_1_1/block5_conv3/Relu:0' shape=(?, ?, ?, 512) dtype=float32>,
 <tf.Tensor 'model_2_1/proposal_layer_1/GatherV2_3:0' shape=(?, 4) dtype=float32>,
 <tf.Tensor 'model_2_1/proposal_layer_1/GatherV2_4:0' shape=(?,) dtype=float32>)

In [61]:
x_.shape.as_list()

[None, None, None, 512]

In [72]:
inp = vgg16.input

x_ = backbone(inp)

x_1, x_2 = rpn(x_)
x_ = RoiAlian()([x_,x_1])
# models.Model(inp, x_)


head = Head(len(LEVEL_1_LABEL))

bbox, score = head(x_)

models.Model(inp, [bbox,score])

ValueError: The shape of the input to "Flatten" is not fully defined (got (None, None, 512). Make sure to pass a complete "input_shape" or "batch_input_shape" argument to the first layer in your model.

In [101]:
rpn.__class__

keras.engine.training.Model

In [99]:
layers.Flatten()(roi)

<tf.Tensor 'flatten_20/Reshape:0' shape=(?, ?) dtype=float32>

In [90]:
head = Head(len(LEVEL_1_LABEL))
head()

(<tf.Tensor 'dense_32/BiasAdd:0' shape=(?, 1772) dtype=float32>,
 <tf.Tensor 'dense_31/Softmax:0' shape=(?, 443) dtype=float32>)

In [86]:
class Head:
    def __init__(self, n_classes):
        self.n_classes = n_classes
        

    def __call__(self, x):
        # x (batch_size=1, rois, pool_width, pool_height, channels)
        x_shape = x.shape.as_list()

#         inp = layers.Input(x_shape[1:])

        x_ = layers.Flatten()(x)
        x_ = layers.Dense(4096, activation='relu',
                    kernel_regularizer=regularizers.l2(1.0),
                    bias_regularizer=regularizers.l2(2.0))(x_)
        x_ = layers.Dense(4096, activation='relu',
                    kernel_regularizer=regularizers.l2(1.0),
                    bias_regularizer=regularizers.l2(2.0))(x_)
        
        score = layers.Dense(self.n_classes, activation='softmax',
                    kernel_initializer=initializers.random_normal(stddev=0.01))(x_)    
        bbox = layers.Dense(self.n_classes*4, activation='linear',
                    kernel_initializer=initializers.random_normal(stddev=0.01))(x_)
#         head_model = models.Model(inp, [bbox, score])
#         return head_model
        return bbox, score

In [63]:
image[0,:,:,0]

array([[ 0.,  1.,  2.,  3.,  4.],
       [ 5.,  6.,  7.,  8.,  9.],
       [10., 11., 12., 13., 14.],
       [15., 16., 17., 18., 19.],
       [20., 21., 22., 23., 24.]], dtype=float32)

In [93]:
image = np.arange(25).astype('float32').reshape(5, 5)
image = image[None,:,:,None]
boxes = np.asarray([[0, 0, 4, 4]], dtype=int)

In [6]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    sess.run(c)
    print(c.eval())

[0 1 2 3]


In [8]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'import tensorflow as tf\ntf.',
  'import tensorflow as tf\ntf.keras',
  'global()',
  'global',
  'globals()',
  'for a in globals():\n    print(a)',
  'globals(0)',
  'globals()'],
 '_oh': {2: <module 'tensorflow._api.v1.keras' from '/home/wcw/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/_api/v1/keras/__init__.py'>,
  5: {...}},
 '_dh': ['/home/wcw/Documents/py-faster-rcnn/keras-faster-rcnn'],
 'In': ['',
  'import tensorflow as tf\ntf.',
  'import tensorflow as tf\ntf.keras',
  'global()',
  'global',
  'globals()',
  'for a in globals():\n    print(a)',
  'globals(0)',
  'globals()'],
 'Out': {2: <module 'tensorflow._api.v1.keras' from '/home/wcw/anaconda3/envs/tf/lib/python3.6/site-packages/tens

In [2]:
import tensorflow as tf
tf.Assert()


<module 'tensorflow._api.v1.keras' from '/home/wcw/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/_api/v1/keras/__init__.py'>